In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
from plotly.offline import iplot
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update({'font.size': 11})

import warnings 
warnings.simplefilter('ignore')
import pickle

with open('../Имена_скважин_pickle/dict_name_well.pickle', 'rb') as f:
    dict_name_well = pickle.load(f)
with open('../Имена_скважин_pickle/dict_name_well_more_365.pickle', 'rb') as f:
    dict_name_well_more_365 = pickle.load(f)
with open('../Имена_скважин_pickle/dict_name_well_reverse.pickle', 'rb') as f:
    dict_name_well_reverse = pickle.load(f)
with open('../Имена_скважин_pickle/dict_name_well_oil_more_365.pickle', 'rb') as f:
    dict_name_well_oil_more_365 = pickle.load(f)

    %config InlineBackend.figure_formats = 'svg'

In [112]:
gtm = pd.read_csv('../../data/Исходные_данные/gtm.csv')
gtm['WELL_NAME'] = gtm['WELL_NAME'].map(dict_name_well_oil_more_365)
gtm.dropna(inplace=True)
gtm.reset_index(inplace=True, drop=True)

with open('../../project/Имена_скважин_pickle/dict_name_well_oil_more_365.pickle', 'wb') as f:
    pickle.dump(dict_name_well_oil_more_365, f)

In [113]:
gtm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   WELL_NAME        225 non-null    float64
 1   GEOTECH_ACTION   225 non-null    object 
 2   START_DATE_FACT  225 non-null    object 
 3   END_DATE_FACT    225 non-null    object 
dtypes: float64(1), object(3)
memory usage: 7.2+ KB


In [19]:
train_oil_year_accum = pd.read_csv('../../data/Преобразованные_данные/train_oil_year_accum.csv')
train_oil_year_accum.head()

,MEASURED_IN_DATE,WELL_NAME,CHARWORK,WATER_CUT,P_ZAB,INTAKE,WELLHEAD_PRESSURE,LIQ_RATE,OIL_RATE,WATER_RATE,Q_LIQ_ACCUM,Q_OIL_ACCUM,Q_WATER_ACCUM
0,2020-01-17,1,НЕФ,26.0,59.20,0.0,NaN,221.0,163.54,57.46,221.0,163.54,57.46
1,2020-01-18,1,НЕФ,26.0,59.20,0.0,NaN,221.0,163.54,57.46,442.0,327.08,114.92
2,2020-01-19,1,НЕФ,26.0,59.20,0.0,NaN,221.0,163.54,57.46,663.0,490.62,172.38
3,2020-01-20,1,НЕФ,26.0,59.20,0.0,NaN,221.0,163.54,57.46,884.0,654.16,229.84
4,2020-01-21,1,НЕФ,30.0,59.26,0.0,NaN,208.0,145.60,62.40,1092.0,799.76,292.24


In [117]:
data_measured = train_oil_year_accum[['MEASURED_IN_DATE', 'WELL_NAME']]
data_measured['MEASURED_IN_DATE'] = pd.to_datetime(data_measured['MEASURED_IN_DATE'])

df_measured = pd.DataFrame()

for well in data_measured['WELL_NAME'].unique():
    data = pd.DataFrame()
    start_date = data_measured[data_measured['WELL_NAME'] == well]['MEASURED_IN_DATE'].min()
    end_date = data_measured[data_measured['WELL_NAME'] == well]['MEASURED_IN_DATE'].max()
    
    data['MEASURED_IN_DATE'] = pd.date_range(start=start_date, 
                                             end=end_date, 
                                             freq='D')
    data['WELL_NAME'] = well
    data = data[['WELL_NAME', 'MEASURED_IN_DATE']]
    df_measured = pd.concat([df_measured, data])
df_measured.reset_index(drop=True, inplace=True)
df_measured

,WELL_NAME,MEASURED_IN_DATE
0,1,2020-01-17
1,1,2020-01-18
2,1,2020-01-19
3,1,2020-01-20
4,1,2020-01-21
...,...,...
125367,258,2022-05-27
125368,258,2022-05-28
125369,258,2022-05-29
125370,258,2022-05-30


In [118]:
data_gtm = gtm[['WELL_NAME', 'START_DATE_FACT', 'END_DATE_FACT']]
data_gtm['START_DATE_FACT'] = pd.to_datetime(data_gtm['START_DATE_FACT'])
data_gtm['END_DATE_FACT'] = pd.DataFrame(data_gtm['END_DATE_FACT'])

df_gtm = pd.DataFrame()

for i in range(data_gtm.shape[0]):
    data = pd.DataFrame()
    well = data_gtm['WELL_NAME'].iloc[i]
    start_date, end_date = data_gtm['START_DATE_FACT'].iloc[i], data_gtm['END_DATE_FACT'].iloc[i]
    

    data['GTM_IN_DATE'] = pd.date_range(start=start_date, 
                                        end=end_date, 
                                        freq='D')
    data['WELL_NAME'] = well
    data = data[['WELL_NAME', 'GTM_IN_DATE']]
    df_gtm = pd.concat([df_gtm, data])
     
df_gtm.reset_index(drop=True, inplace=True)    
df_gtm

,WELL_NAME,GTM_IN_DATE
0,148.0,2020-10-03
1,148.0,2020-10-04
2,148.0,2020-10-05
3,148.0,2020-10-06
4,235.0,2020-07-01
...,...,...
2722,37.0,2016-12-13
2723,37.0,2016-12-14
2724,37.0,2016-12-15
2725,37.0,2016-12-16


In [17]:
from functions.make_df_without_gaps import make_df_without_gaps

#make_df_without_gaps(dataframe=train_oil_year_accum, date_column='MEASURED_IN_DATE', group_params='WELL_NAME')

In [18]:
make_df_without_gaps(dataframe=train_oil_year_accum, date_column='MEASURED_IN_DATE', group_params='WELL_NAME')

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [122]:
df_gtm.rename(columns={'GTM_IN_DATE': 'DATE'}, inplace=True)
df_measured.rename(columns={'MEASURED_IN_DATE': 'DATE'}, inplace=True)


merge_df = pd.merge(df_measured, df_gtm, on=['WELL_NAME', 'DATE'], how='outer', indicator=True)
merge_df[merge_df['_merge'] == 'both']

,WELL_NAME,DATE,_merge
0,1.0,2020-01-17,both
1338,4.0,2019-10-09,both
1339,4.0,2019-10-10,both
1340,4.0,2019-10-11,both
1341,4.0,2019-10-12,both
...,...,...,...
121024,249.0,2021-03-31,both
121648,250.0,2020-11-04,both
121649,250.0,2020-11-05,both
121650,250.0,2020-11-06,both


merge_df.to_csv('../../data/Преобразованные_данные/dates_intersection_of_gtm_and_measured', index=False)